In [25]:
from statsbombpy import sb
import pandas as pd
import numpy as np

In [26]:
#Para ver resultados de copa america (son datos gratuitos)
df = sb.matches(competition_id=11, season_id=27)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [27]:
df.columns

Index(['match_id', 'match_date', 'kick_off', 'competition', 'season',
       'home_team', 'away_team', 'home_score', 'away_score', 'match_status',
       'match_status_360', 'last_updated', 'last_updated_360', 'match_week',
       'competition_stage', 'stadium', 'referee', 'home_managers',
       'away_managers', 'data_version', 'shot_fidelity_version',
       'xy_fidelity_version'],
      dtype='object')

In [28]:
teams1 = df['home_team'].unique()
teams2 = df['away_team'].unique()

In [29]:
teams = set(teams1) | set(teams2)

In [55]:
def get_team_id_matches(team, df):
    idvalues = df[(df['home_team'] == team) | (df['away_team'] == team)]['match_id'].unique()
    return idvalues

In [56]:
def get_players(team, df):
    #given a team, it will return its players
    idvalues = get_team_id_matches(team, df)
    players = set()
    for id in idvalues:
        lineup = sb.lineups(match_id=id)[team]
        players.update(lineup['player_name'].unique())
    #print(players)
    return players

In [57]:
lineups = {}

for team in teams:
    lineups[f'lineup_{team}'] = set() 
    lineups[f'lineup_{team}'].update(get_players(team, df=df))  

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [58]:
lineups

{'lineup_Atlético Madrid': {'Amath Ndiaye',
  'Antoine Griezmann',
  'Augusto Matías Fernández',
  'Claudio Matías Kranevitter',
  'Diego Roberto Godín Leal',
  'Fernando José Torres Sanz',
  'Filipe Luís Kasmirski',
  'Gabriel Fernández Arenas',
  'Guilherme Magdalena Siqueira',
  'Ignacio Monsalve Vicente',
  'Jackson Arley Martínez Valencia',
  'Jan Oblak',
  'Jesús Gámez Duarte',
  'Jorge Resurrección Merodio',
  'José María Giménez de Vargas',
  'Juan Francisco Torres Belén',
  'Lucas Hernández Pi',
  'Luciano Vietto',
  'Miguel Ángel Moyà Rumbo',
  'Raúl García Escudero',
  'Samuel Villa Rodríguez',
  'Saúl Ñíguez Esclapez',
  'Stefan Savić',
  'Theo Bernard François Hernández',
  'Thomas Teye Partey',
  'Tiago Cardoso Mendes',
  'Yannick Ferreira Carrasco',
  'Ángel Martín Correa',
  'Óliver Torres Muñoz'},
 'lineup_Getafe': {'Abdoul Karim Yoda',
  'Alexis Ruano Delgado',
  'Balázs Megyeri',
  'Bernard Mensah',
  'Borja Galán González',
  'Carlos Martín Vigaray',
  'Damián Nicol

In [78]:
sb.events(match_id=3825882)

HTTPError: 500 Server Error: Internal Server Error for url: https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/3825882.json

In [73]:
def calculate_passes_ij(player1, player2, team_id):
    #calculates probability of pass from player 1 to player 2 based on an estimator
    total_proportion = 0
    match_count = 0

    for id in team_id:
        #relevant events (pass)
        events = sb.events(match_id=id)
        pass_events = events[events['type']=='Pass']
        
        #relevant passes (players)
        pass_events_ij = pass_events[(pass_events['player'] == player1) & (pass_events['pass_recepient'] == player2)]
        total_i_pass = pass_events[pass_events['player']==player1]

        #length
        passes_ij = len(pass_events_ij)
        passes_i = len(total_i_pass)

        if passes_i > 0:
            proportion = passes_ij / passes_i
        else:
            proportion = 0
        
        total_proportion+=proportion

        match_count+=1

    return total_proportion/match_count

In [77]:
get_team_id_matches('Atlético Madrid',df)

array([3825882, 3825860, 3825890, 3825838, 3825834, 3825828, 3825907,
       3825813, 3825807, 3825784, 3825795, 3825787, 3825642, 3825638,
       3825607, 3825579, 3825774, 3825711, 3825701, 3825693, 3825682,
       3825674, 3825666, 3825658, 3825598, 3825878, 3825872, 3825760,
       3825748, 3825738, 3825728, 3825714, 3825631, 3825621, 3825593,
       3825563,  267576,  266166])